In [ ]:
import zipfile, os
local_zip = '/content/drive/MyDrive/Doggy.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall()
zip_ref.close()

In [ ]:
print("Jumlah Data Train Tiap Kelas")
print('Jumlah gambar golden retriever :', len(os.listdir('/content/Doggy/Golden Retriever dog')))
print('Jumlah gambar poodle           :', len(os.listdir('/content/Doggy/Poodle dog')))
print('Jumlah gambar samoyed          :', len(os.listdir('/content/Doggy/Samoyed dog')))
print('Jumlah gambar shih             :', len(os.listdir('/content/Doggy/Shih tzu dog')))
print('Jumlah gambar siberian         :', len(os.listdir('/content/Doggy/Siberian Husky dog')))

Jumlah Data Train Tiap Kelas
Jumlah gambar golden retriever : 167
Jumlah gambar poodle           : 242
Jumlah gambar samoyed          : 185
Jumlah gambar shih             : 200
Jumlah gambar siberian         : 152


In [ ]:
os.mkdir('/tmp/s/')
os.mkdir('/tmp/s/train/')
os.mkdir('/tmp/s/val/')
os.mkdir('/tmp/s/train/golden/')
os.mkdir('/tmp/s/val/golden/')
os.mkdir('/tmp/s/train/poodle/')
os.mkdir('/tmp/s/val/poodle/')
os.mkdir('/tmp/s/train/samoyed/')
os.mkdir('/tmp/s/val/samoyed/')
os.mkdir('/tmp/s/train/shih/')
os.mkdir('/tmp/s/val/shih/')
os.mkdir('/tmp/s/train/siberian/')
os.mkdir('/tmp/s/val/siberian/')

In [ ]:
import random
from shutil import copyfile

def train_val_split(source, train, val, train_ratio):
  total_size = len(os.listdir(source))
  train_size = int(train_ratio * total_size)
  val_size = total_size - train_size

  randomized = random.sample(os.listdir(source), total_size)
  train_files = randomized[0:train_size]
  val_files = randomized[train_size:total_size]

  for i in train_files:
    i_file = source + i
    destination = train + i
    copyfile(i_file, destination)

  for i in val_files:
    i_file = source + i
    destination = val + i
    copyfile(i_file, destination)

train_ratio = 0.8

#Pembagian Training dan Validasi

source_00 = '/content/Doggy/Golden Retriever dog/'
train_00 = '/tmp/s/train/golden/'
val_00 = '/tmp/s/val/golden/'
train_val_split(source_00, train_00, val_00, train_ratio)

source_01 = '/content/Doggy/Poodle dog/'
train_01 = '/tmp/s/train/poodle/'
val_01 = '/tmp/s/val/poodle/'
train_val_split(source_01, train_01, val_01, train_ratio)

source_02 = '/content/Doggy/Samoyed dog/'
train_02 = '/tmp/s/train/samoyed/'
val_02 = '/tmp/s/val/samoyed/'
train_val_split(source_02, train_02, val_02, train_ratio)

source_01 = '/content/Doggy/Shih tzu dog/'
train_01 = '/tmp/s/train/shih/'
val_01 = '/tmp/s/val/shih/'
train_val_split(source_01, train_01, val_01, train_ratio)

source_02 = '/content/Doggy/Siberian Husky dog/'
train_02 = '/tmp/s/train/siberian/'
val_02 = '/tmp/s/val/siberian/'
train_val_split(source_02, train_02, val_02, train_ratio)

In [ ]:

print('Jumlah All golden     :', len(os.listdir('/content/Doggy/Golden Retriever dog/')))
print('Jumlah Train golden   :', len(os.listdir('/tmp/s/train/golden')))
print('Jumlah Val golden     :', len(os.listdir('/tmp/s/val/golden')))

Jumlah All golden     : 167
Jumlah Train golden   : 133
Jumlah Val golden     : 34


In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(
                  rescale = 1./255.,
                  rotation_range = 30,
                  horizontal_flip = True,
                  shear_range = 0.3,
                  fill_mode = 'nearest',
                  width_shift_range = 0.2,
                  height_shift_range = 0.2,
                  zoom_range = 0.1
)

val_datagen = ImageDataGenerator(
                rescale = 1./255.,
                rotation_range = 30,
                horizontal_flip = True,
                shear_range = 0.3,
                fill_mode = 'nearest',
                width_shift_range = 0.2,
                height_shift_range = 0.2,
                zoom_range = 0.1
)
     

In [ ]:
train_dir = '/tmp/s/train/'
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (200, 200),
    batch_size = 3,
    class_mode = 'categorical'
)

val_dir = '/tmp/s/val/'
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size = (200, 200),
    batch_size = 3,
    class_mode = 'categorical'
)

Found 755 images belonging to 5 classes.
Found 191 images belonging to 5 classes.


In [ ]:
# callbacks
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs = {}):
    if(logs.get('accuracy') > 0.99):
      print('\nAkurasi mencapai 99%')
      self.model.stop_training = True

callbacks = myCallback()

In [ ]:
model = tf.keras.models.Sequential([
          tf.keras.layers.Conv2D(8, (3, 3), activation = 'relu', input_shape = (200, 200, 3)),
          tf.keras.layers.MaxPooling2D(2, 2),
          tf.keras.layers.Conv2D(16, (3, 3), activation = 'relu'),
          tf.keras.layers.MaxPooling2D(2, 2),
          tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'),
          tf.keras.layers.MaxPooling2D(2, 2),
          tf.keras.layers.Flatten(),
          tf.keras.layers.Dense(500, activation = 'relu'),         
          tf.keras.layers.Dense(5, activation = 'softmax')
])

In [ ]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_28 (Conv2D)          (None, 198, 198, 8)       224       
                                                                 
 max_pooling2d_21 (MaxPoolin  (None, 99, 99, 8)        0         
 g2D)                                                            
                                                                 
 conv2d_29 (Conv2D)          (None, 97, 97, 16)        1168      
                                                                 
 max_pooling2d_22 (MaxPoolin  (None, 48, 48, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_30 (Conv2D)          (None, 46, 46, 32)        4640      
                                                                 
 max_pooling2d_23 (MaxPoolin  (None, 23, 23, 32)      

In [ ]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'Adam',
              metrics = ['accuracy'])

In [ ]:
history = model.fit(
            train_generator,
            steps_per_epoch = 20,
            epochs = 25,
            validation_data = val_generator,
            validation_steps = 5,
            verbose = 1,
            callbacks = [callbacks]
)

Epoch 1/25
20/20 [==============================] - 4s 166ms/step - loss: 2.3026 - accuracy: 0.1667 - val_loss: 2.0492 - val_accuracy: 0.2000
Epoch 2/25
20/20 [==============================] - 3s 160ms/step - loss: 1.6956 - accuracy: 0.3167 - val_loss: 1.6065 - val_accuracy: 0.2667
Epoch 3/25
20/20 [==============================] - 3s 167ms/step - loss: 1.6127 - accuracy: 0.2000 - val_loss: 1.5730 - val_accuracy: 0.4000
Epoch 4/25
20/20 [==============================] - 3s 148ms/step - loss: 1.6109 - accuracy: 0.2333 - val_loss: 1.5698 - val_accuracy: 0.4667
Epoch 5/25
20/20 [==============================] - 3s 145ms/step - loss: 1.6207 - accuracy: 0.2667 - val_loss: 1.5852 - val_accuracy: 0.2667
Epoch 6/25
20/20 [==============================] - 3s 160ms/step - loss: 1.5908 - accuracy: 0.3167 - val_loss: 1.6119 - val_accuracy: 0.2667
Epoch 7/25
20/20 [==============================] - 3s 133ms/step - loss: 1.5390 - accuracy: 0.3333 - val_loss: 1.8154 - val_accuracy: 0.2000
Epoch 

In [ ]:
%matplotlib inline

import matplotlib.image  as mpimg
import matplotlib.pyplot as plt


acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label = 'Training Accuracy')
plt.plot(epochs, val_acc, 'b', label = 'Validation Accuracy')
plt.title('Training and Validation accuracy')
plt.legend(loc = 'best')
plt.show()

plt.plot(epochs, loss, 'r', label = 'Training Loss')
plt.plot(epochs, val_loss, 'b', label = 'Validation Loss')
plt.title('Training and Validation accuracy')
plt.legend(loc = 'best')
plt.show()

In [ ]:
import numpy as np
from keras.preprocessing import image
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():

  # predicting images
  path = fn
  img = tf.keras.utils.load_img(path, target_size = (150, 150))
  imgplot = plt.imshow(img)
  x = tf.keras.utils.img_to_array(img)
  x = np.expand_dims(x, axis = 0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size = 100)

  print(fn)

  class_list = os.listdir('/content/Doggy/')
  
  for j in range(42):
    if classes[0].any() :
      print('This image belongs to class', class_list[0])
      break

In [ ]:
import numpy as np
from keras.preprocessing import image
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():

  # predicting images
  path = fn
  img = tf.keras.utils.load_img(path, target_size = (150, 150))
  imgplot = plt.imshow(img)
  x = tf.keras.utils.img_to_array(img)
  x = np.expand_dims(x, axis = 0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size = 100)

  print(fn)

  class_list = os.listdir('/content/Doggy/')
  
  for j in range(42):
    if classes[0].any() :
      print('This image belongs to class', class_list[0])
      break

In [ ]:
from google.colab import drive
drive.mount('/content/drive')